# VGG 16 - 4 Emotions Notebook

### Import Packages 

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model
from keras.preprocessing import image

In [27]:
import numpy as np
from numpy.random import randn
import pathlib
import random
from PIL import Image
from matplotlib.image import imread
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/PranavM98/Facial-Detection-and-Emotion-Classifier.git

Cloning into 'Facial-Detection-and-Emotion-Classifier'...
remote: Enumerating objects: 87229, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 87229 (delta 10), reused 16 (delta 7), pack-reused 87206
Receiving objects: 100% (87229/87229), 340.90 MiB | 33.66 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Checking out files: 100% (134147/134147), done.


In [39]:
data_dir = pathlib.Path("/content/Facial-Detection-and-Emotion-Classifier/00_Data/01_Transformed_Data/Buckets_Resized_Emotion_Images/")
data=pd.read_csv("/content/Facial-Detection-and-Emotion-Classifier/00_Data/01_Transformed_Data/buckets_emotion_label.csv")

In [40]:
img_data=[]
all_images = list(data_dir.glob('*'))

for i in all_images:
  img=Image.open(str(i))
  img=img.resize((64,64))
  img=np.array(img).astype('float64')
  img=img/255
  img_data.append(img)


In [41]:
all_labels=[]
label_names={'Happy':0,
            'Sad':1,
            'Neutral':2,
            'Angry':3}

for i in all_images:
  file=str(i)[str(i).rfind('/')+1:]
  all_labels.append(label_names[data[data['Filename']==file]['Label'].values[0]])

### Masking Pre-Processing

In [42]:
top_mask=[]
bottom_mask=[]

def creating_mask():
    a=np.array([0]*2048)
    a=a.reshape((32,64))
    b=np.array([1]*2048)
    b=b.reshape((32,64))
    return a,b


for i in img_data:
    # a,b=creating_mask()
    # con_bot = np.concatenate((b,a),axis=0)
    # con_top = np.concatenate((a,b),axis=0)
    # bottom_mask.append(np.multiply(con_bot,i))
    # top_mask.append(np.multiply(con_top,i))
    bottom_mask.append(i[:32,:])
    top_mask.append(i[32:,:])

In [43]:
#Bottom Mask
data_size=len(all_images)

train_test_split=(int)(data_size*0.2)

x_train_bm=bottom_mask[train_test_split:]
x_test_bm=bottom_mask[:train_test_split]

y_train_bm=all_labels[train_test_split:]
y_test_bm=all_labels[:train_test_split]




In [44]:
train_dataset_bm = tf.data.Dataset.from_tensor_slices((np.asarray(x_train_bm), np.array(y_train_bm)))
test_dataset_bm = tf.data.Dataset.from_tensor_slices((np.asarray(x_test_bm), np.array(y_test_bm)))

In [45]:
#Top Mask
data_size=len(all_images)

train_test_split=(int)(data_size*0.2)

x_train_tm=top_mask[train_test_split:]
x_test_tm=top_mask[:train_test_split]

y_train_tm=all_labels[train_test_split:]
y_test_tm=all_labels[:train_test_split]



In [46]:
train_dataset_tm = tf.data.Dataset.from_tensor_slices((np.asarray(x_train_tm), np.array(y_train_tm)))
test_dataset_tm = tf.data.Dataset.from_tensor_slices((np.asarray(x_test_tm), np.array(y_test_tm)))

### Baseline No Mask Pre-Processing

In [58]:
data_size=len(all_images)

train_test_split=(int)(data_size*0.2)

x_train=img_data[train_test_split:]
x_test=img_data[:train_test_split]

y_train=all_labels[train_test_split:]
y_test=all_labels[:train_test_split]

In [59]:
for i in range(len(x_train)):
    x_train[i]=x_train[i].reshape((64,64,1))

for i in range(len(x_test)):
    x_test[i]=x_test[i].reshape((64,64,1))

In [60]:
train_dataset = tf.data.Dataset.from_tensor_slices((np.asarray(x_train), np.array(y_train)))
test_dataset = tf.data.Dataset.from_tensor_slices((np.asarray(x_test), np.array(y_test)))

### Validation

In [61]:
len(x_train)

28710

In [62]:
len(x_train_tm)

28710

In [63]:
len(x_train_bm)

28710

## VGG-16 Model

In [64]:
device_name = tf.test.gpu_device_name()

input = Input(shape=(64,64,1))

x=Conv2D(filters=64,kernel_size=3, padding='same',activation='relu')(input)
x=Conv2D(filters=64,kernel_size=3, padding='same',activation='relu')(x)
x=MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

x=Conv2D(filters=128,kernel_size=3, padding='same', activation='relu')(x)
x=Conv2D(filters=128,kernel_size=3, padding='same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding= 'same')(x)

x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)

x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)

x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)

x=Flatten()(x)
x=Dense(units=4096, activation='relu')(x)
x=Dense(units=4096, activation='relu')(x)
x=Dense(units=1000, activation='relu')(x)
output=Dense(units=3, activation='softmax')(x)

In [65]:
model=Model(inputs=input, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 1)]       0         
                                                                 
 conv2d_39 (Conv2D)          (None, 64, 64, 64)        640       
                                                                 
 conv2d_40 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 conv2d_42 (Conv2D)          (None, 32, 32, 128)       147584    
                                                           

In [66]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.00001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=tf.keras.metrics.SparseCategoricalAccuracy()
)

In [67]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [68]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6)
model.fit(train_dataset, epochs=25, validation_data=(test_dataset),callbacks=[callback])

Epoch 1/25
449/449 [==============================] - 27s 57ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 2/25
449/449 [==============================] - 25s 57ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 3/25
449/449 [==============================] - 25s 56ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 4/25
449/449 [==============================] - 25s 56ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 5/25
449/449 [==============================] - 25s 56ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 6/25
449/449 [==============================] - 25s 56ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - 

In [69]:
model.save("/content/Facial-Detection-and-Emotion-Classifier/10_Code/03_Model_Checkpoints/four_buckets_no_mask_model")

INFO:tensorflow:Assets written to: /content/Facial-Detection-and-Emotion-Classifier/10_Code/03_Model_Checkpoints/four_buckets_no_mask_model/assets


In [70]:
y_pred=model.predict(np.asarray(x_test))
y_preds=np.argmax(y_pred,axis=1)

In [71]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report

In [72]:
confusion_matrix(y_test,y_preds)

array([[2589,    0,    0,    0],
       [2235,    0,    0,    0],
       [1240,    0,    0,    0],
       [1113,    0,    0,    0]])

In [73]:
accuracy_score(y_test,y_preds)

0.36073568343318935

In [74]:
f1_score(y_test,y_preds,average='weighted')

0.1912645268090369

In [75]:
print(classification_report(y_test,y_preds))

              precision    recall  f1-score   support

           0       0.36      1.00      0.53      2589
           1       0.00      0.00      0.00      2235
           2       0.00      0.00      0.00      1240
           3       0.00      0.00      0.00      1113

    accuracy                           0.36      7177
   macro avg       0.09      0.25      0.13      7177
weighted avg       0.13      0.36      0.19      7177



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Bottom Mask

In [76]:
train_dataset_bm = train_dataset_bm.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset_bm = test_dataset_bm.batch(BATCH_SIZE)

In [77]:
input = Input(shape=(32,64,1))
x=Conv2D(filters=64,kernel_size=3, padding='same',activation='relu')(input)
x=Conv2D(filters=64,kernel_size=3, padding='same',activation='relu')(x)
x=MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
x=Conv2D(filters=128,kernel_size=3, padding='same', activation='relu')(x)
x=Conv2D(filters=128,kernel_size=3, padding='same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding= 'same')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)

x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)


x=Flatten()(x)
x=Dense(units=4096, activation='relu')(x)
x=Dense(units=4096, activation='relu')(x)
x=Dense(units=1000, activation='relu')(x)
output=Dense(units=3, activation='softmax')(x)

model_bm=Model(inputs=input, outputs=output)
model_bm.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 64, 1)]       0         
                                                                 
 conv2d_52 (Conv2D)          (None, 32, 64, 64)        640       
                                                                 
 conv2d_53 (Conv2D)          (None, 32, 64, 64)        36928     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 16, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 16, 32, 128)       73856     
                                                                 
 conv2d_55 (Conv2D)          (None, 16, 32, 128)       147584    
                                                           

In [78]:
model_bm.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=tf.keras.metrics.SparseCategoricalAccuracy()
)


In [79]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model_bm.fit(train_dataset_bm, epochs=25, validation_data=(test_dataset_bm),callbacks=[callback])

Epoch 1/25
449/449 [==============================] - 19s 39ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 2/25
449/449 [==============================] - 17s 38ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 3/25
449/449 [==============================] - 17s 38ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607


In [80]:
model_bm.save("/content/Facial-Detection-and-Emotion-Classifier/10_Code/03_Model_Checkpoints/four_buckets_bottom_mask_model")

INFO:tensorflow:Assets written to: /content/Facial-Detection-and-Emotion-Classifier/10_Code/03_Model_Checkpoints/four_buckets_bottom_mask_model/assets


In [81]:
y_pred=model_bm.predict(np.asarray(x_test_bm))
y_preds=np.argmax(y_pred,axis=1)

In [82]:
confusion_matrix(y_test_bm,y_preds)

array([[2589,    0,    0,    0],
       [2235,    0,    0,    0],
       [1240,    0,    0,    0],
       [1113,    0,    0,    0]])

In [83]:
accuracy_score(y_test_bm,y_preds)

0.36073568343318935

In [84]:
f1_score(y_test_bm,y_preds, average='weighted')

0.1912645268090369

In [85]:
print(classification_report(y_test_bm,y_preds))

              precision    recall  f1-score   support

           0       0.36      1.00      0.53      2589
           1       0.00      0.00      0.00      2235
           2       0.00      0.00      0.00      1240
           3       0.00      0.00      0.00      1113

    accuracy                           0.36      7177
   macro avg       0.09      0.25      0.13      7177
weighted avg       0.13      0.36      0.19      7177



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TOP Mask

In [86]:
train_dataset_tm = train_dataset_tm.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset_tm = test_dataset_tm.batch(BATCH_SIZE)

In [87]:
input = Input(shape=(32,64,1))
x=Conv2D(filters=64,kernel_size=3, padding='same',activation='relu')(input)
x=Conv2D(filters=64,kernel_size=3, padding='same',activation='relu')(x)
x=MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
x=Conv2D(filters=128,kernel_size=3, padding='same', activation='relu')(x)
x=Conv2D(filters=128,kernel_size=3, padding='same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding= 'same')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)

x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu')(x)
x=MaxPool2D(pool_size=2, strides=2, padding='same')(x)


x=Flatten()(x)
x=Dense(units=4096, activation='relu')(x)
x=Dense(units=4096, activation='relu')(x)
x=Dense(units=1000, activation='relu')(x)
output=Dense(units=3, activation='softmax')(x)


model_tm=Model(inputs=input, outputs=output)
model_tm.summary()


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 64, 1)]       0         
                                                                 
 conv2d_65 (Conv2D)          (None, 32, 64, 64)        640       
                                                                 
 conv2d_66 (Conv2D)          (None, 32, 64, 64)        36928     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 16, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_67 (Conv2D)          (None, 16, 32, 128)       73856     
                                                                 
 conv2d_68 (Conv2D)          (None, 16, 32, 128)       147584    
                                                           

In [88]:
model_tm.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=tf.keras.metrics.SparseCategoricalAccuracy()
)


In [89]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model_tm.fit(train_dataset_tm, epochs=25, validation_data=(test_dataset_tm),callbacks=[callback])

Epoch 1/25
449/449 [==============================] - 18s 38ms/step - loss: nan - sparse_categorical_accuracy: 0.3620 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 2/25
449/449 [==============================] - 17s 38ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607
Epoch 3/25
449/449 [==============================] - 17s 38ms/step - loss: nan - sparse_categorical_accuracy: 0.3623 - val_loss: nan - val_sparse_categorical_accuracy: 0.3607


In [90]:
model_tm.save("/content/Facial-Detection-and-Emotion-Classifier/10_Code/03_Model_Checkpoints/four_buckets_top_mask_model")

INFO:tensorflow:Assets written to: /content/Facial-Detection-and-Emotion-Classifier/10_Code/03_Model_Checkpoints/four_buckets_top_mask_model/assets


In [91]:
y_pred=model_tm.predict(np.asarray(x_test_tm))
y_preds=np.argmax(y_pred,axis=1)

In [92]:
confusion_matrix(y_test_tm,y_preds)

array([[2589,    0,    0,    0],
       [2235,    0,    0,    0],
       [1240,    0,    0,    0],
       [1113,    0,    0,    0]])

In [93]:
accuracy_score(y_test_tm,y_preds)

0.36073568343318935

In [94]:
f1_score(y_test_tm,y_preds, average='weighted')

0.1912645268090369

In [95]:
print(classification_report(y_test_tm,y_preds))

              precision    recall  f1-score   support

           0       0.36      1.00      0.53      2589
           1       0.00      0.00      0.00      2235
           2       0.00      0.00      0.00      1240
           3       0.00      0.00      0.00      1113

    accuracy                           0.36      7177
   macro avg       0.09      0.25      0.13      7177
weighted avg       0.13      0.36      0.19      7177



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
